In [ ]:
# !git clone https://github.com/quantanty/ml-pytorch-sklearn-book.git
# %cd /content/ml-pytorch-sklearn-book/
# !git fetch
# !git checkout practice/ch15-B
# %cd ch15

fatal: destination path 'ml-pytorch-sklearn-book' already exists and is not an empty directory.
/content/ml-pytorch-sklearn-book
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 412 bytes | 412.00 KiB/s, done.
From https://github.com/quantanty/ml-pytorch-sklearn-book
   0000213..58d2fd5  practice/ch15-B -> origin/practice/ch15-B
Already on 'practice/ch15-B'
Your branch is behind 'origin/practice/ch15-B' by 1 commit, and can be fast-forwarded.
  (use "git pull" to update your local branch)
/content/ml-pytorch-sklearn-book/ch15


In [ ]:
# %cd /content/ml-pytorch-sklearn-book/
# !git pull origin practice/ch15-B
# %cd ch15

/content/ml-pytorch-sklearn-book
From https://github.com/quantanty/ml-pytorch-sklearn-book
 * branch            practice/ch15-B -> FETCH_HEAD
Updating 0000213..58d2fd5
Fast-forward
 ch15/utils/model.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)
/content/ml-pytorch-sklearn-book/ch15


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn

device = torch.device('cuda')

In [1]:
import importlib
import utils
importlib.reload(utils)

<module 'utils' from '/home/quan-pham/Documents/Study/MLPytorch/ch15/utils/__init__.py'>

In [3]:
start = 'THE MYSTERIOUS ISLAND'
end = '\n\n*** END OF THE PROJECT GUTENBERG'
# mys_dataset = utils.data.MysteriousIsland('/content/drive/MyDrive/Machine Learning/data/1268-0.txt', 41, start, end)
mys_dataset = utils.data.MysteriousIsland('../data/1268-0.txt', 41, start, end)

In [66]:
train_set, test_set = utils.data.split_dataset(mys_dataset, train_size=0.4)
batch_size = 64
train_dl = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)

In [6]:
vocab_size = len(mys_dataset.int2char)
embed_dim = 256
rnn_hidden_size = 512
torch.manual_seed(555)
model = utils.model.RNN_v1(vocab_size)

In [7]:
# model.load_state_dict(torch.load('/content/gen-v1.pth'))
model.load_state_dict(torch.load('../models/gen-v1.pth'))
model = model.to(device)
model

RNN_v1(
  (embedding): Embedding(79, 256)
  (rnn): LSTM(256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=79, bias=True)
)

In [8]:
from torch.distributions.categorical import Categorical
import numpy as np
c2i = mys_dataset.char2int
i2c = mys_dataset.int2char
encode, decode = mys_dataset.get_encoder_decoder()

def sample(model, starting_str, len_generated_text=500, scale_factor=1.0):
    encoded_input = torch.tensor([c2i[s] for s in starting_str])
    encoded_input = torch.reshape(encoded_input, (1, -1)).to(device)
    generated_str = starting_str

    model.eval()
    hidden, cell = model.init_hidden(1)
    hidden = hidden.to(device)
    cell = cell.to(device)
    for c in range(len(starting_str)-1):
        _, hidden, cell = model(encoded_input[:, c].view(1), hidden, cell)

    last_char = encoded_input[:, -1]
    for i in range(len_generated_text):
        logits, hidden, cell = model(last_char.view(1), hidden, cell)
        logits = torch.squeeze(logits, 0)
        scaled_logits = logits * scale_factor
        m = Categorical(logits=scaled_logits)
        last_char = m.sample()
        generated_str += str(i2c[last_char])

    return generated_str

In [18]:
print(sample(model, starting_str='The mysterious', scale_factor=2))

The mysterious animal, the productions of the first part of the river, and the reporter listened to the long slowly, and the tempest gave a few moments the sailor was probable that some accident
of the powder-magazine. The reporter returned to Granite House.

The settlers had been seen from the sea.

Ayrton to fear the reporter’s companions. The shore was thus face the left bank of the Mercy, and leaving the lake was returned, were to be done but to return to the river, which would be always reached the water


In [ ]:
class RNN_v2(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, rnn_hidden_size=512, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.layer_norm = nn.LayerNorm(2*rnn_hidden_size)
        self.fc = nn.Linear(2*rnn_hidden_size, vocab_size)

    def forward(self, x, hidden, cell):
        out = self.embedding(x).unsqueeze(1)
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.layer_norm(out)
        out = self.fc(out).reshape(out.size(0), -1)
        return out, hidden, cell

    def init_hidden(self, batch_size):
        hidden = torch.zeros(2, batch_size, self.rnn_hidden_size)
        cell = torch.zeros(2, batch_size, self.rnn_hidden_size)
        return hidden, cell

model_2 = RNN_v2(vocab_size)
model_2 = model_2.to(device)
model_2

RNN_v2(
  (embedding): Embedding(79, 128)
  (rnn): LSTM(128, 512, batch_first=True, bidirectional=True)
  (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (fc): Linear(in_features=1024, out_features=79, bias=True)
)

In [ ]:
seq_length = len(mys_dataset.chunks[0]) - 1
def train(model, dl, loss_fn, optimizer, num_epochs):
    milestone_length = 500
    epoch_ranges = [(0, 1), (1, milestone_length)] + [(i, i+milestone_length) for i in range(milestone_length, num_epochs, milestone_length)]

    for i, (start_epoch, end_epoch) in enumerate(epoch_ranges):
        loss: float
        for epoch in range(start_epoch, end_epoch):
            hidden, cell = model.init_hidden(batch_size)
            hidden = hidden.to(device)
            cell = cell.to(device)
            seq_batch, target_batch = next(iter(dl))
            seq_batch = seq_batch.to(device)
            target_batch = target_batch.to(device)
            optimizer.zero_grad()
            loss = 0
            for c in range(seq_length):
                pred, hidden, cell = model(seq_batch[:, c], hidden, cell)
                loss += loss_fn(pred, target_batch[:, c])
            loss.backward()
            optimizer.step()
            loss = loss.item()/seq_length

        print(f'epoch {epoch:4d} loss: {loss}')

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_2.parameters(), lr=0.001)

train(model_2, train_dl, loss_fn, optimizer, 5000)

epoch    0 loss: 4.466468429565429
epoch  499 loss: 1.3365333557128907
epoch  999 loss: 1.2513358116149902
epoch 1499 loss: 1.1688644409179687
epoch 1999 loss: 1.1567015647888184
epoch 2499 loss: 1.1062256813049316
epoch 2999 loss: 1.0191046714782714
epoch 3499 loss: 0.9999540328979493
epoch 3999 loss: 0.9570250511169434
epoch 4499 loss: 0.9450458526611328
epoch 4999 loss: 0.971148681640625


In [74]:
model_2_1 = utils.model.RNN_v2(vocab_size)
model_2_1 = model_2_1.to(device)
model_2_1

RNN_v2(
  (embedding): Embedding(79, 128)
  (rnn): LSTM(128, 512, batch_first=True, bidirectional=True)
  (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (fc): Linear(in_features=1024, out_features=79, bias=True)
)

In [94]:
seq_length = len(mys_dataset.chunks[0]) - 1
def train_batch(model, dl, loss_fn, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        loss = 0
        
        for i, (seq_batch, target_batch) in enumerate(dl):
            hidden, cell = model.init_hidden(dl.batch_size)
            hidden = hidden.to(device)
            cell = cell.to(device)
            seq_batch = seq_batch.to(device)
            target_batch = target_batch.to(device)
            optimizer.zero_grad()
            loss_batch = 0
            for c in range(seq_length):
                pred, hidden, cell = model(seq_batch[:, c], hidden, cell)
                loss_batch += loss_fn(pred, target_batch[:, c])
            loss_batch.backward()
            optimizer.step()
            loss += loss_batch.item() / seq_length

            if i % 500 == 0:
                print(f'batch {i:4d}\tloss: {loss / (i+1)}')
        
        loss /= len(dl)

        print(f'epoch {epoch:4d})\tloss: {loss}')

In [75]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_2_1.parameters(), lr=0.001)

train_batch(model_2_1, train_dl, loss_fn, optimizer, 1)

batch    0	loss: 4.4457141876220705
batch  500	loss: 1.6548116918095566
batch 1000	loss: 1.4747299299135335
batch 1500	loss: 1.3828511717159089
batch 2000	loss: 1.3221959203675304
batch 2500	loss: 1.2779088856934664
batch 3000	loss: 1.2425471702761282
batch 3500	loss: 1.2128794192042032
batch 4000	loss: 1.1872820041144516
batch 4500	loss: 1.164249623274704
batch 5000	loss: 1.1437394311298887
batch 5500	loss: 1.1246847383110483
batch 6000	loss: 1.107233327834773
batch 6500	loss: 1.0909475824719617
epoch    0)	loss: 69.31913240485731


In [86]:
print(sample(model_2_1, "the mysterious", scale_factor=2.0))

the mysterious being, who
uninhabited, of the convicts, boiltry-yard and the apparatus. “We must consider
the expedition to those of the Mercy. It would not be better than two companions arrived at the
bottom of the sea.

“There is no longer any means of the southern point of the
islet, with the aid of the alarm had evidently fallen to the sea the sailor he had not as yet each eight o’clock in the morning, the presence of the promontory, although he shared the colonists, could not see the sailor and Gideon Sp


In [90]:
train_set_2, test_test_2 = utils.data.split_dataset(test_set, train_size=0.2)
train_dl_2 = DataLoader(train_set_2, batch_size, shuffle=True, drop_last=True, num_workers=4)

In [92]:
torch.save(model_2_1.state_dict(), '../models/gen-v2.1.pth')

In [95]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_2_1.parameters(), lr=0.001)

train_batch(model_2_1, train_dl_2, loss_fn, optimizer, 1)

batch    0	loss: 0.9061808586120605
batch  500	loss: 0.8584881321874681
batch 1000	loss: 0.8497989320612099
batch 1500	loss: 0.8413363224502262
batch 2000	loss: 0.8333709676285984
epoch    0)	loss: 0.8321314360483669


In [99]:
print(sample(model_2_1, "clock clock clock clock", scale_factor=2.0))

clock clock clock clocked balls through the forest, the sea having been convinced that the engineer had not
been deceived until he might have been able to discover the aspect of the islet at the mouth of Falls River. There they began to come without their having had any idea of the colonists would have been made to him.

“Gentlemen,” he said, “Herbert, who was then lighted up the ladder.”

“Spilett,” answered the engineer.

“Quite my opinion.”

“And when you have said that the intelligent animal works cast with the se


In [100]:
torch.save(model_2_1.state_dict(), '../models/gen-v2.2.pth')

In [103]:
model_2 = utils.model.RNN_v2(vocab_size)
model_2.load_state_dict(torch.load('../models/gen-v2.2.pth'))
model_2 = model_2.to(device)
model_2

RNN_v2(
  (embedding): Embedding(79, 128)
  (rnn): LSTM(128, 512, batch_first=True, bidirectional=True)
  (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (fc): Linear(in_features=1024, out_features=79, bias=True)
)

In [104]:
print(sample(model_2, "clock clock clock clock", scale_factor=2.0))

clock clock clock clocks. A few trees, where the boat was not in the same material.

It was the stranger was ready, and which might be left for at any cost to a thousand questions of the engineer.

“Well, captain,” replied the sailor, “there is the matter, the return of day, and also that the declivity of the animal would be
sufficient to convey it to the coast of the island was rather, the case was employed in the western shore of Lake Grant; worked with a powerful being, and it was agreed that the settlers in Lincol


In [114]:
from tqdm import tqdm

def eval(model, dl, loss_fn):
    model.eval()
    loss = 0
    with torch.no_grad():
        for seq_batch, target_batch in tqdm(dl):
            hidden, cell = model.init_hidden(dl.batch_size)
            hidden = hidden.to(device)
            cell = cell.to(device)
            seq_batch = seq_batch.to(device)
            target_batch = target_batch.to(device)
            loss_batch = 0
            for c in range(seq_length):
                pred, hidden, cell = model(seq_batch[:, c], hidden, cell)
                loss_batch += loss_fn(pred, target_batch[:, c])
            loss += loss_batch.item() / seq_length
    
    return loss / len(dl)

In [115]:
test_dl = DataLoader(test_test_2, batch_size, shuffle=None, drop_last=True, num_workers=4)
test_loss = eval(model_2, test_dl, loss_fn)
test_loss

100%|██████████| 8341/8341 [04:07<00:00, 33.76it/s]


0.8036209054994238